In [1]:
import numpy as np

In [2]:
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t') #按制表符分割
        dataMat.append([float(lineArr[0]),float(lineArr[1])]) #list扩充一个新的list
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat #返回的数据和标签是list

In [3]:
def selectJrand(i,m):
    j = i
    while (j == i):
        j = int(np.random.uniform(0,m))
        #np.random.uniform(low,high,size) 从一个均匀分布[low,high)中随机采样
    return j
# A value is randomly chosen and return as long as it's not equal to the input i

In [4]:
def clipAlpha(aj,H,L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj #alpha 取值有上下限，如果大于上限H，则取H；如果小于下限L，则取L

In [5]:
dataArr,labelArr = loadDataSet('testSet.txt')

In [6]:
from time import time

In [7]:
def smoSimple(dataMatIn,classLabels,C,toler,maxIter):
    t0 = time()
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()
    b = 0
    m,n = dataMatrix.shape
    alphas = np.mat(np.zeros((m,1)))
    iters = 0
    while iters < maxIter:
        alphaPairsChanged = 0
        for i in range(m):
            #预测的类别
            fXi = float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b  
            #f(x)=E a(i)*y(i)<xi,x> + b
            #np.multiply() 对应元素相乘
            Ei = fXi - float(labelMat[i]) #误差
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m) #randomly select second alpha
                fXj = float(np.multiply(alphas,labelMat).T * (dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                #python passes all lists by reference,so you have to explicitly tell python to give you a new memory location for alphaIold
                #and alphaJold.Otherwise,when you later compare the new and old values,we won't see the change.
                alphaJold = alphas[j].copy()
                #如果y(i)与y(j)异/同号,alpha[j]的边界
                if (labelMat[i] != labelMat[j]):
                    L = max(0,alphas[j]-alphas[i])
                    H = min(C,C+alphas[j]-alphas[i])
                else:
                    L = max(0,alphas[j]+alphas[i]-C)
                    H = min(C,alphas[j]+alphas[i])
                if L==H: 
                    print ('L==H')
                    continue #跳出此次的循环并开始下一个值的循环
                eta = 2.0*dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                # calculate eta(the similarity of sample i and j);the second derivative of the objective function along the diagonal line 
                # smo  eta=k(x1,x1)+k(x2,x2)-2k(x1,x2) 理论上这个值一般大于0
                # 此处代码相当于理论公式的-eta
                if eta >= 0:
                    print ('eta>=0')
                    continue
                # update alpha[j]
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)
                if (np.abs(alphas[j] - alphaJold) < 0.00001):
                    print ('J not moving enough')
                    continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])
                # update i by the same amount as j in opposite direction
                b1 = b - Ei - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - \
                    labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - \
                    labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0<alphas[i]) and (C>alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1+b2)/2.0
                alphaPairsChanged += 1
                print ('iters: %d i:%d,pair changed %d' %(iters,i,alphaPairsChanged))
        if alphaPairsChanged == 0: 
            iters += 1
        else:
            iters = 0
        print('iteration number: %d' % (iters))
    print("done in %0.3fs" % (time()-t0))
    return b,alphas      

In [12]:
b,alphas = smoSimple(dataArr,labelArr,0.6,0.001,40)

iters: 0 i:0,pair changed 1
iters: 0 i:2,pair changed 2
L==H
L==H
iters: 0 i:17,pair changed 3
J not moving enough
J not moving enough
L==H
J not moving enough
L==H
iters: 0 i:52,pair changed 4
L==H
iters: 0 i:55,pair changed 5
J not moving enough
L==H
J not moving enough
L==H
L==H
J not moving enough
L==H
L==H
J not moving enough
L==H
iteration number: 0
J not moving enough
J not moving enough
J not moving enough
L==H
L==H
J not moving enough
J not moving enough
L==H
J not moving enough
J not moving enough
L==H
L==H
J not moving enough
iters: 0 i:23,pair changed 1
iters: 0 i:29,pair changed 2
J not moving enough
J not moving enough
J not moving enough
J not moving enough
L==H
J not moving enough
J not moving enough
iteration number: 0
J not moving enough
J not moving enough
J not moving enough
J not moving enough
J not moving enough
J not moving enough
J not moving enough
iters: 0 i:55,pair changed 1
iters: 0 i:57,pair changed 2
J not moving enough
L==H
L==H
J not moving enough
J not 

In [13]:
b

matrix([[-3.83674285]])

In [14]:
alphas[alphas>0]

matrix([[ 0.16768456,  0.12179207,  0.08077279,  0.37024941]])

In [15]:
for i in range(100):
    if alphas[i]>0.0:
        print (dataArr[i],labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[2.893743, -1.643468] -1.0
[6.080573, 0.418886] 1.0


In [41]:
#创建一个对象保存用到的变量
# I could have done this just as easily with Python dictionary,but that takes more work trying to access member variables
class optStruct(object):
    def __int__(self,dataMatIn,classLabels,C,toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m,1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m,2)))
        # all of these are the same as in the simplified SMO code,but you've added the member variable eCache
        #eCache:the first column is a flag bit stating whether the eCache is valid（有效的）,and the second column is the actual E value

In [44]:
def calcEK(oS,k):
    fXk = float(np.multiply(oS.alphas,oS.labelMat).T*(oS.X*oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJ(i,oS,Ei):
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]
    validEcacheList = np.nonzero(oS.eCache[:,0].A)[0]
    # .A : change the data type from a matrix to an array
    #nonzero() return a list containing indices of the input list that are not zero
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            if k == i : 
                continue
            Ek = calcEk(oS,k)
            deltaE = abs(Ei-Ek)
            if (deltaE > maxDeltaE):
                maxK = k;maxDeltaE=deltaE;Ej=Ek
        return maxK,Ej
    else:
        j = selectJrand(i,oS.m)
        Ej = calcEk(oS,j)
    return j,Ej
def updateEk(oS,k):
    Ek = calcEk(oS,k)
    oS.eCache[k] = [1,Ek]

In [45]:
def innerL(i,oS):
    Ei = calcEk(oS,i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i,oS,Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0,oS.alphas[j]-oS.alphas[i])
            H = min(oS.C,oS.C+oS.alphas[j]-oS.alphas[i])
        else:
            L = max(0,oS.alphas[j]+oS.alphas[i]-oS.C)
            H = min(oS.C,oS.alphas[j]+oS.alphas[i])
        if L==H:
            print('L==H')
            return 0
        eta = 2.0*oS.X[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - oS.X[j,:]*oS.X[j,:].T
        if eta >= 0:
            print('eta>=0')
            return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei-Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        
        if (abs(oS.alphas[j]-alphaJold) < 0.00001):
            print('J not moving enough')
            return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold-oS.alphas[j])
        updateEk(oS,i)
        b1 = oS.b - Ei - oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS[i,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
        b2 = oS.b - Ej - oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS[j,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T
        if (0<oS.alphas[i]) and (oS.C>oS.alphas[i]):
            oS.b = b1
        elif (0<os.alphas[j]) and (oS.C>oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1+b2)/2.0
        return 1
    else:
        return 0

In [46]:
def smoP(dataMatIn,classLabels,C,toler,maxIter,kTup=('lin',0)):
    oS = optStruct(np.mat(dataMatIn),np.mat(classLabels).transpose(),C,toler)
    iters = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter<maxIter) and ((alphaPairsChanged>0) or (entireSet)):
        alphaPairsChaged = 0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
            print('fullSet,iters:%d i:%d,pairs changed %d' %(iters,i,alphaPairsChanged))
            iters += 1
        else:
            nonBoundIs = nonzero((oS.alphas.A>0)*(Os.alphas.A<C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print('non-bound,iters:%d i:%d,pairs changed %d' %(iters,i,alphaPairsChanged))
            iters += 1
        if entireSet:
            entireSet = False
        elif(alphaPairsChanged==0):
            entireSet = True
        print('iteration number:%d' %(iters))
    return oS.b,oS.alphas

In [47]:
dataArr,labelArr = loadDataSet('testSet.txt')

In [48]:
smoP(dataArr,labelArr,0.6,0.0001,40)

TypeError: object() takes no parameters